In [7]:
#Utilizacion de una GPU si esta disponible
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [8]:
#Instalacion de pytorch
!pip install pytorch-pretrained-bert pytorch-nlp

In [9]:
#Imports necesarios para la ejecucion del codigo
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [10]:
#Preparacion de la GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

In [11]:
#Subimos los dos archivos necesarios para trabajar. Train y Test
from google.colab import files
uploaded = files.upload()

Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [12]:
#Creacion de los dos data frame
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test.drop(columns=['location', 'keyword'], inplace=True)


In [13]:
#Imports necesarios para limpiar correctamente el codigo
import pandas as pd
pd.set_option('max_colwidth', -1)
import string
import nltk
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#Al igual que en otros notebooks, preprocesamiento previo
# Quitamos las urls
df['text'] = df['text'].str.replace(r'http:\/\/.*', '', regex=True).replace(r'https:\/\/.*', '', regex=True)

# Quitamos user mentions, signos de puntuación, hashtags y stopwords.
def clean_text(text):
    words = text.lower().split(' ')
    text = ' '.join([word for word in words if not word.startswith('@') and word not in stopwords.words('english')])
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

df['text'] = df['text'].apply(clean_text)

In [14]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse into nearby homes http://t.co/STfMbbZFB5,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control wild fires in California even in the Northern part of the state. Very troubling.,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.,1


In [15]:
#Bert necesita las oraciones que componen el texto de cada tweet pre-procesados de forma especial. Asi que extraemos los campos de oraciones de cada entrada
sentences = df.text.values

#Y les agregamos un separador antes y despues.
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = df.target.values

In [16]:
import torch

#Codigo para informar que se esta utilizando correctamente la GPU.
if torch.cuda.is_available():        
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [17]:
#Utilizaremos el BERT pre entrenado para minusculas (Por eso, realizamos un preprocesamiento previo que hemos repetido en otros notebooks inicial del texto)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

#Luego, separaremos las oraciones en lo que se conoce como tokens. Cada palabra equivale a un token
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

100%|██████████| 231508/231508 [00:00<00:00, 2609385.39B/s]


Tokenize the first sentence:
['[CLS]', 'our', 'deeds', 'are', 'the', 'reason', 'of', 'this', '#', 'earthquake', 'may', 'allah', 'forgive', 'us', 'all', '[SEP]']


In [18]:
#Establecemos la cantidad maxima de palabras que puede tener un tweet. Con 128 nos aseguramos de que no haya ningun tweet que sea cortado
MAX_LEN = 128

In [19]:
#Se obtienen los respectivos id de cada una de las series de tokens que representa un texto.
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [20]:
#Se necesita que todos los id, tengan el tamaño maximo, asi que les aplicaremos padding en caso de ser necesario
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [21]:
# Es necesario crear una mascara para cada id
attention_masks = []

# De igual manera, es necesario aplicar padding de ser necesario
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [22]:
# Particion del test en set de entrenamiento y de validacion

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [23]:
# Ultima conversion de la data para que sea utilizada por BERT

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [24]:
# Los autores recomiendan que esta variable sea 16 o 32
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [25]:

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

100%|██████████| 407873900/407873900 [00:08<00:00, 50945997.37B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [26]:
#Hiperparametros posibles y la obtencion de los optimos
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [27]:
# optimizer tiene toda la informacion requerida sobre los hiperparametros
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [28]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [32]:
t = [] 

# Otra variable que los autores recomiendan que este entre 2 y 4
epochs = 4


for _ in trange(epochs, desc="Epoch"):
  
  
  # Entrenamiento
  
  # Es necesario setear el modelo como entrenamiento
  model.train()
  
  # Variables de seguimiento
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  

  for step, batch in enumerate(train_dataloader):
    # Para utilizacion de GPU
    batch = tuple(t.to(device) for t in batch)
    # Desempacamos del iterador
    b_input_ids, b_input_mask, b_labels = batch
    #Con estas lineas, reseteamos el gradiente entre variables y podemos calcular la perdida de informacion al entrenar
    optimizer.zero_grad()
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels) 
    loss.backward()
    # Actualizacion de la variable con hiperparametros
    optimizer.step()
    
    
    #Actualizacion de las variables de seguimiento
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validacion

  # Cambio de modo
  model.eval()

  # Nuevas variables de seguimiento 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluamos por cada una de las iteraciones que decidimos anteriormente
  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    # Se recomienda la siguiente linea para optimizar tiempos
    with torch.no_grad():
      #Calculo de predicciones
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Train loss: 0.43964921073165053


Epoch:  25%|██▌       | 1/4 [05:25<16:17, 325.72s/it]

Validation Accuracy: 0.8669871794871794
Train loss: 0.2884586351901986


Epoch:  50%|█████     | 2/4 [10:50<10:50, 325.42s/it]

Validation Accuracy: 0.8594751602564102
Train loss: 0.1700322621003833


Epoch:  75%|███████▌  | 3/4 [16:14<05:25, 325.08s/it]

Validation Accuracy: 0.8344350961538461
Train loss: 0.10338959320080142


Epoch: 100%|██████████| 4/4 [21:38<00:00, 324.74s/it]

Validation Accuracy: 0.8214142628205128


In [ ]:
df_test
#Al igual que en otros notebooks, preprocesamiento previo
# Quitamos las urls
df_test['text'] = df_test['text'].str.replace(r'http:\/\/.*', '', regex=True).replace(r'https:\/\/.*', '', regex=True)

# Quitamos user mentions, signos de puntuación, hashtags y stopwords.
def clean_text(text):
    words = text.lower().split(' ')
    text = ' '.join([word for word in words if not word.startswith('@') and word not in stopwords.words('english')])
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

df_test['text'] = df_test['text'].apply(clean_text)

In [33]:


#Realizamos el mismo preprocesamiento
sentences = df_test.text.values

#Sin embargo, por razones obvias, no vamos a obtener los labels
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128


input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []


for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [34]:
# Modelo en modo evaluacion
model.eval()

#Variable para guardar las predicciones
predictions = []

# Predecir
for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask = batch
  with torch.no_grad():
    #Calculo de las predicciones
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  logits = logits.detach().cpu().numpy()
  
  #Guardamos las predicciones
  predictions.append(logits)

In [35]:
#Las predicciones no se guardan de forma "normal" en un vector, asi que es necesario reacomodarlas
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
len(flat_predictions)

3263

In [36]:
df_test['target'] = flat_predictions
df_test

,id,text,target
0,0,Just happened a terrible car crash,1
1,2,"Heard about #earthquake is different cities, stay safe everyone.",1
2,3,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",1
3,9,Apocalypse lighting. #Spokane #wildfires,1
4,11,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTENERS XrWn,1
3259,10865,Storm in RI worse than last hurricane. My city&amp;3others hardest hit. My yard looks like it was bombed. Around 20000K still without power,1
3260,10868,Green Line derailment in Chicago http://t.co/UtbXLcBIuY,1
3261,10874,MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3,1


In [37]:
df_test[['id', 'target']].to_csv('bert2.csv',index=False)